In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

fairuzazaria_rtm_stuck_prediction_datasets_path = kagglehub.dataset_download('fairuzazaria/rtm-stuck-prediction-datasets')

print('Data source import complete.')


# **DATA GATHERING**

## **1. PREPARATION**

### 1.0. Install Required Libraries

In [ ]:
!pip install fastparquet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 29.2 MB/s eta 0:00:0000:01


### 1.1 Import Libraries

In [ ]:
import os, csv, glob
import math, pyarrow
import datetime, fastparquet

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import pyarrow.parquet as pq
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

In [ ]:
from tqdm.notebook import tqdm
from multiprocessing import Lock

tqdm.set_lock(Lock())

### 1.2 Create Required Functions

In [ ]:
def get_dataset_directories(pattern: str = 'txt', base: str = '/kaggle/input') -> tuple:
    working_dir = glob.glob(os.path.join(base, '*'), recursive=True)
    dataset_dir = tuple(
        tuple(
            filename for filename in glob.iglob(os.path.join(dir, '**', f'*{pattern}'), recursive=True)
        )
        for dir in working_dir
    )

    return (dataset_dir, working_dir)

In [ ]:
def get_missing_timestamps(dataframe: pd.DataFrame, date_column: str, freq: str = '5s') -> tuple:
    missing = pd.date_range(
        start = dataframe[date_column].min(),
        end   = dataframe[date_column].max(),
        freq  = freq
    )

    return tuple(missing.difference(dataframe[date_column]))

In [ ]:
def set_content_csv(data: list, method: str = 'a') -> None:
    if (os.path.exists(path_destination) == False):
        method = 'w'

    with open(path_destination, method, newline='') as csvfile:
        writer = csv.writer(csvfile)
        if (method == 'w'):
            writer.writerow(data[0])

        del data[0]
        writer.writerows(data)

In [ ]:
def set_data_label(df: pd.DataFrame, date_range: tuple, date_col: str = 'Date-Time', label_col: str = 'Stuck', label: int = 1) -> None:
    dataframe = df.copy()
    for start, end in tqdm(date_range):
        dataframe.loc[dataframe[date_col].between(start, end), label_col] = label

    dataframe = dataframe.fillna(0)
    return dataframe

## **2. DATA GATHERING**

In [ ]:
#-- get all directories
directories = get_dataset_directories(pattern = "csv")
parent_dir  = directories[1]
dataset_dir = directories[0][0]

print(f'fetched {len(dataset_dir)} well d data!')

fetched 11 well d data!


In [ ]:
#-- read datasets
df_combined = pd.concat([pd.read_csv(directory) for directory in dataset_dir], axis=0, ignore_index=True)

df_combined['dt'] = pd.to_datetime(df_combined['dt'])
df_combined = df_combined.sort_values(by=['dt'])
df_combined = df_combined.reset_index(drop=True)

print(f'fetched well d with {df_combined.shape} shape!')

fetched well d with (523744, 13) shape!


In [ ]:
#-- adjust dataset
features = [
    "dt", "blockpos",
    "bitdepth", "md",
    "hklda", "mudflowin",
    "rop", "rpm", "torqa",
    "stppress", "woba"
]

df_combined = df_combined[features]
df_combined = df_combined.drop_duplicates() #-- remove duplicates
df_combined[df_combined.columns[1:]] = df_combined[df_combined.columns[1:]].astype(float) #-- change datatype
df_combined[df_combined.columns[1:]] = df_combined[df_combined.columns[1:]].clip(lower=0) #-- clip negatives

df_combined.tail()

,dt,blockpos,bitdepth,md,hklda,mudflowin,rop,rpm,torqa,stppress,woba
523739,2024-06-11 23:59:37,5.18,0.0,1753.1,34.63,0.0,20.0,0.0,0.0,0.0,0.0
523740,2024-06-11 23:59:42,5.18,0.0,1753.1,34.69,0.0,20.0,0.0,0.0,0.0,0.0
523741,2024-06-11 23:59:47,5.18,0.0,1753.1,34.44,0.0,20.0,0.0,0.0,0.0,0.0
523742,2024-06-11 23:59:52,5.18,0.0,1753.1,34.32,0.0,20.0,0.0,0.0,0.0,0.0
523743,2024-06-11 23:59:57,5.18,0.0,1753.1,34.69,0.0,20.0,0.0,0.0,0.0,0.0


In [ ]:
#-- dataset info
print(f'well d shape      : {df_combined.shape}')
print(f'well d missing    : {len(get_missing_timestamps(df_combined, df_combined.columns[0]))}')
print(f'well d uplicates  : {df_combined["dt"].duplicated().sum()}')
print(f'well d nan values : \n{df_combined.isna().sum()}\n')

well d shape      : (523739, 11)
well d missing    : 253757
well d uplicates  : 0
well d nan values : 
dt           0
blockpos     0
bitdepth     0
md           0
hklda        0
mudflowin    0
rop          0
rpm          0
torqa        0
stppress     0
woba         0
dtype: int64



In [ ]:
#-- store stuck events
# stuck_d = [(datetime.datetime(2024, 6, 7, 10, 0),  datetime.datetime(2024, 6, 7, 20, 26, 15))]
stuck_d = [(datetime.datetime(2024, 6, 7, 10, 39),  datetime.datetime(2024, 6, 7, 20, 4, 40))]

In [ ]:
#-- add label
df_combined = set_data_label(df_combined, stuck_d, date_col = "dt", label_col = "stuck", label = 1)

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
#-- resample to 5s
df_combined  = df_combined.set_index("dt")
df_resampled = df_combined[df_combined.columns[:-1]].resample("5s").interpolate()
df_resampled["stuck"] = df_combined['stuck'].resample("5s").ffill()

df_resampled = df_resampled.reset_index()
df_combined  = df_resampled

del df_resampled

df_combined.head()

,dt,blockpos,bitdepth,md,hklda,mudflowin,rop,rpm,torqa,stppress,woba,stuck
0,2024-05-12 13:30:20,30.39,0.0,0.01,49.60,0.0,0.0,0.0,0.04,0.0,0.0,0.0
1,2024-05-12 13:30:25,30.39,0.0,0.01,49.52,0.0,0.0,0.0,0.02,0.0,0.0,0.0
2,2024-05-12 13:30:30,30.39,0.0,0.01,49.60,0.0,0.0,0.0,0.02,0.0,0.0,0.0
3,2024-05-12 13:30:35,30.39,0.0,0.01,49.52,0.0,0.0,0.0,0.02,0.0,0.0,0.0
4,2024-05-12 13:30:40,30.39,0.0,0.01,49.44,0.0,0.0,0.0,0.01,0.0,0.0,0.0


In [ ]:
#-- dataset info
print(f'well d shape      : {df_combined.shape}')
print(f"well d labels     : {list(df_combined['stuck'].value_counts())}")
print(f'well d missing    : {len(get_missing_timestamps(df_combined, df_combined.columns[0]))}')
print(f'well d uplicates  : {df_combined["dt"].duplicated().sum()}')
print(f'well d nan values : \n{df_combined.isna().sum()}\n')

well d shape      : (525956, 12)
well d labels     : [519168, 6788]
well d missing    : 0
well d uplicates  : 0
well d nan values : 
dt           0
blockpos     0
bitdepth     0
md           0
hklda        0
mudflowin    0
rop          0
rpm          0
torqa        0
stppress     0
woba         0
stuck        0
dtype: int64



## **3. SAVE DATASET**

In [ ]:
df_combined.to_parquet(
    os.path.join(os.getcwd(), f"well_d.parquet"),
    engine = "pyarrow",
    compression = "snappy"
)